In [1]:
import requests
import pandas as pd
import numpy as np
import json
import os
import glob
import re
import time
from collections import Counter
from IPython.core.debugger import set_trace

from tqdm.auto import tqdm
tqdm.pandas()

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

C:\Users\infomax\Documents\GitHub\getch\venv\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import django
from django.db import connection
django.setup()

from getchapp.models import CustomEmailUser, Brand, Item, Profile, Post

# DB 초기화

In [3]:
def get_logo_fname(bname, where):
    try:
        return glob.glob(where + '/' + bname + '.*')[0].split('\\')[-1]
    except:
        return 'need file.jpg'
    
get_logo_fname('helinox', 'uploads/brand_images')

'helinox.png'

In [4]:
#CustomEmailUser.objects.all().delete();
# Profile.objects.all().delete();
# Hashtag.objects.all().delete();
BrandChannel.objects.all().delete();
# Feed.objects.all().delete();
# Channel.objects.all().delete();
ItemChannel.objects.all().delete();

# Brand

In [21]:
'''
이렇게 하면, model에서 save() 함수 override 한게 안먹힌다 (2019.11.21)
'''

# bulk_brand = []
ibrands = list(pd.read_excel('keywords and logos.xlsx', sheet_name='20190430').itertuples())[3:]

for i, row in enumerate(tqdm(sorted(ibrands, key=lambda x:x.shortname)[::-1])):
    obj = Brand()
    obj.id = i+1
    obj.name = row.shortname
    obj.keywords = row.keywords
    
    obj.fullname_kr = row.koname
    obj.fullname_en = row.fullname
    obj.origin = row.origin
    #obj.awareness = row.awareness
    obj.category = row.category.lower()
    obj.description = ''
    obj.image = os.path.join('brand_images', get_logo_fname(row.shortname, 'uploads/brand_images'))
    obj.save()
    #bulk_brand.append(obj)
    
#Brand.objects.bulk_create(bulk_brand);

In [4]:
ibrands = list(pd.read_excel('keywords and logos.xlsx', sheet_name='20190430').itertuples())[3:]

for row in tqdm(sorted(ibrands, key=lambda x:x.shortname)[::-1]):
    opt = {
        #'id': id+1,
        'name': row.shortname,
        'keywords': row.keywords,
        'fullname_kr': row.koname,
        'fullname_en': row.fullname,
        'origin': row.origin,
        'category': row.category.lower(),
        'description': '',
        'image': os.path.join('brand_images', get_logo_fname(row.shortname, 'uploads/brand_images'))
    }
    Brand.objects.create(**opt)

# Item

In [5]:
items = [
    {'name': 'skirt', 'keywords': 'skirt, skirts, 스커트, 치마', 'image':'item_images/skirt.png'},
    {'name': 'shoe', 'keywords': 'shoe, shoes, 슈즈, 신발', 'image':'item_images/shoe.png'},
    {'name': 'bottom', 'keywords': 'bottom, bottoms, 하의', 'image':'item_images/bottom.png'},
    {'name': 'outer', 'keywords': 'outer, outers, 아우터', 'image':'item_images/outer.png'},
    {'name': 'top', 'keywords': 'top, tops, 상의', 'image':'item_images/top.png'},
    {'name': 'dress', 'keywords': 'dress, 드레스, 원피스', 'image':'item_images/dress.png'},
    {'name': 'bag', 'keywords': 'bag, bags, 백, 가방', 'image':'item_images/bag.png'},
    {'name': 'accessory', 'keywords': 'accessory, accessories, 악세서리, 장신구', 'image':'item_images/accessory.png'},
]

for item in tqdm(items):
    opt = {
        'name': item['name'],
        'keywords': item['keywords'],
        'image': item['image'],
    }
    Item.objects.create(**opt)

# Post

In [4]:
profile = Profile.objects.get(pk=4); profile

<Profile: sejin76@nate.com>

In [5]:
posts = [
    {'image':'post_images/samples/1.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/2.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/3.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/4.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/5.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/6.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/7.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/8.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/9.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/10.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/11.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/12.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/13.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/14.jpg', 'text':'111111111111'},
    {'image':'post_images/samples/15.jpg', 'text':'111111111111'},
]

for post in tqdm(posts):
    opt = {
        'author': profile,
        'image': post['image'],
        'text': post['text'],
    }
    Post.objects.create(**opt)

# Channel

In [7]:
bulk_channel = []
for i, br in enumerate(tqdm(bulk_brand)):
    obj = Channel()
    obj.id = i+1
    obj.content = br
    bulk_channel.append(obj)
    
Channel.objects.bulk_create(bulk_channel);

# CustomEmailUser

In [8]:
users = None
with open('users.json', encoding='UTF-8-sig') as f:
    users = json.load(f)

In [9]:
bulk_emailuser = []
for _id, _user in tqdm(users.items()):
    obj = CustomEmailUser()
    obj.id = _id
    obj.email = _user['user'] + '@getch.com'
    bulk_emailuser.append(obj)
    
CustomEmailUser.objects.bulk_create(bulk_emailuser);

# Profile

In [10]:
bulk_profile = []
for i, (_id, _user) in enumerate(tqdm(list(users.items())[:])):
    obj = Profile()
    obj.id = _id
    obj.user = bulk_emailuser[i]
    obj.image = 'profile_images/' + _user['user'] + '@getch.com' + '/' + _user['profile_image'].split('/')[-1]
    bulk_profile.append(obj)
    
Profile.objects.bulk_create(bulk_profile);
# Profile.objects.bulk_update(bulk_profile, ['image']);

# Hashtag

In [11]:
hashtags = None
with open('hashtags.json', encoding='UTF-8-sig') as f:
    hashtags = json.load(f)

In [12]:
EMOJI = r'[\U00010000-\U0010ffff]'

In [13]:
bulk_hashtags = []

for i, _tag in enumerate(tqdm(hashtags)):
    obj = Hashtag()
    obj.id = i+1
    _tag = re.sub(EMOJI, '', _tag) # MySQL에서는 이모티콘 저장이 잘 안된다. 아예 지워버리자...
    
    if _tag != '':
        obj.hashtag = _tag
        bulk_hashtags.append(obj)
        
Hashtag.objects.bulk_create(bulk_hashtags);

# Feed

In [14]:
channels_dict = {ch.content.name:ch.pk for ch in bulk_channel}

In [15]:
profile_dict = {pr.user.email:pr for pr in bulk_profile}

In [16]:
hashtags_dict = {ht.hashtag:ht.pk for ht in bulk_hashtags}

In [17]:
feeds = None
with open('feeds.json', encoding='UTF-8-sig') as f:
    feeds = json.load(f)

In [18]:
bulk_feeds = []

for i, _feed in enumerate(tqdm(feeds[:])):
    obj = Feed()
    obj.id = i+1
    # obj.membership = brand_dict[_feed['membership']] # 이렇게 foreignkey가 들어가려면, 해당 필드의 id가 있어야 하는듯 하다 (확인은 안됨)
    obj.author = profile_dict[_feed['author']+'@getch.com']
    obj.timestamp = str(pd.Timestamp(_feed['created_at']).tz_localize(None))
    obj.nlikes = _feed['nlikes']
    obj.content = re.sub(EMOJI, '', _feed['content'])
    
    if 'feed_image' in _feed:
        # _feed[membership] 부분은, page가 여러개 있는 경우 - 로 연결하는 걸로: nike-adidas-custompage 등
        obj.image = 'feed_images/' + _feed['membership'] + '/' + _feed['author'] + '@getch.com' + '/' + _feed['feed_image'].split('/')[-1]
    
    bulk_feeds.append(obj)

In [19]:
# Feed.objects.bulk_update(bulk_feeds, ['timestamp']);
# Feed.objects.bulk_update(bulk_feeds, ['image']);
Feed.objects.bulk_create(bulk_feeds);

In [20]:
FeedHashtagsRelation = Feed.hashtags.through

relations = []
for i, _feed in enumerate(tqdm(feeds)):
    for ht in _feed['hashtags']:
        ht = re.sub(EMOJI, '', ht)
        if ht != '':
            relations.append(FeedHashtagsRelation(feed_id=i+1, hashtag_id=hashtags_dict[ht]))
            
FeedHashtagsRelation.objects.bulk_create(relations);

In [21]:
FeedChannelsRelation = Feed.channels.through

relations = []
for i, _feed in enumerate(tqdm(feeds)):
    relations.append(FeedChannelsRelation(feed_id=i+1, channel_id=channels_dict[_feed['membership']]))
            
FeedChannelsRelation.objects.bulk_create(relations);

In [9]:
class TEST:
    pass

class KKK(TEST):
    def __init__(self):
        print(self.__class__.__name__)
        return None

In [11]:
KKK()

KKK
